In [17]:
import numpy as np
import scipy as sp

In [18]:
import scipy.sparse as sparse

In [33]:
a = np.array([1,2,3])

def insert(arr, index, value):
    if index > len(arr):
        new_arr = np.zeros((index + 1,))
        new_arr[:len(arr)] = arr
        arr = new_arr
    arr[index] = value
    return arr
        
insert(a, 10, 7)


array([1., 2., 3., 0., 0., 0., 0., 0., 0., 0., 7.])

In [31]:
np.concatenate(([1], np.arange(5, 50 + 5, 5)))

array([ 1,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [35]:
np.tile(np.arange(2, 7), 4)

array([2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6])

In [ ]:
# define a range of learning rates to test
alphas = np.arange(0.05, 1 + 0.05, 0.05)
# define a range of softmax parameters to test
betas = np.concatenate(([1], np.arange(5, 50 + 5, 5)))
# define a range of WM reliance to test
rhos = np.arange(0, 1 + 0.05, 0.05)
# define a range of capacities to test
Ks = np.arange(2, 6 + 1)

# for a finer grid:
# alphas = [.06:.01:.5];%[0.05:.05:1];
# betas = [1 4:2:20];%[1 5:5:50];
# rhos = [.5:.01:.98];%[0:.05:1];


# define simulation parameters
realalpha = .1
realbeta = 10
realrho = .9
realK = 4
## simulate the RLWM task

b = 0
t = 0
update = np.zeros(dtype=int)


Reps = 3
Trials = 15
nsMin = 2
nsMax = 6

alloc = Reps * Trials * (nsMax - nsMin + 1) / 2 * ( 2 * nsMax + (nsMax - nsMin))
update = np.zeros((1, alloc))
choice = np.zeros((1, alloc))

for rep in range(Reps): # three repetitions of each set size
    for ns in range(nsMin, nsMax + 1): # block of set size ns
        
        b = b + 1
        update[t] = 1
        # initialize WM mixture weight
        w = realrho * np.min(1, realK / ns)
        # initialize RL and WM agents
        Q = .5 + np.zeros(ns, 3)
        WM = .5 + np.zeros(ns, 3)
        # define a sequence of trials with 15 iterations of each stimulus
        trials = np.tile(np.array(1, ns + 1, 1), Trials)
        # loop over trials
        for s in trials:
            t = t + 1
            # RL policy
            softmax1 = np.exp(realbeta * Q[s - 1, :]) / np.sum(np.exp(realbeta * Q[s - 1, :]))
            # WM policy
            softmax2 = np.exp(50 * WM[s - 1, :]) / np.sum(np.exp(50 * WM[s - 1, :]))
            # mixture policy
            pr = (1 - w) *softmax1 + w * softmax2
            # action choice
            r = rand()
            if r < pr[0]:
                choice[t] = 1
            elif r < pr[0] + pr[1]
                choice[t] = 2 
            else
                choice[t] = 3
           
            # reward correct action (arbitrarily defined here)
            rew[t] = choice[t] == (rem(s,3)+1);
            # update RL and WM agents
            Q(s,choice(t))=Q(s,choice(t))+realalpha*(rew(t)-Q(s,choice(t)));
            WM(s,choice(t))=rew(t);
            # store information
            stim(t)=s;
            setsize(t)=ns;
        end
    end
end